In this step, new features summarizing previous games performance will be generated

In [1]:
import pandas as pd
#solicitando info de un website
import requests
import sys
sys.path.append("/Users/kike/opt/anaconda3/lib/python3.9/site-packages/")
import bs4
pd.pandas.set_option ("display.max_columns", None)
import ssl
import math 
import time
import random
import re
import pathlib

#reading csv
path = pathlib.Path.cwd() / 'sets' / 'mlb_dataset.csv'
df_viejo = pd.read_csv(path, index_col = None)

#Making corrections to teams names that have changed 
df_viejo.loc[df_viejo["Local"] == "Indians",  "Local"] = "Guardians"
df_viejo.loc[df_viejo["Visit"] == "Indians",  "Visit"] = "Guardians"

#Making corrections on similar teams names
df_viejo.loc[df_viejo["Local"] == "Red",  "Local"] = "R.Sox"
df_viejo.loc[df_viejo["Visit"] == "Red",  "Visit"] = "R.Sox"

df_viejo.head(50)

,Fecha,Local,Visit,Local Score,Visit Score,Local i5,Visit i5,Local Hits,Visit Hits,Local pitcher,Local pitcher in,Local pitcher ERA,Local pitcher Hit ERA,Local pitcher K ERA,Local pitcher BB ERA,Visit pitcher,Visit pitcher in,Visit pitcher ERA,Visit pitcher Hit ERA,Visit pitcher K ERA,Visit pitcher BB ERA,Local Win?,L bullpen recieved,V bullpen recieved
0,20150405,Cubs,Cardinals,0.0,3.0,0.0,3.0,5.0,10.0,J. Lester,4.1,3.0,8.0,6.0,2.0,A. Wainwright,6.0,0.0,5.0,6.0,0.0,0.0,0.0,0.0
1,20150406,Yankees,Blue,1.0,6.0,0.0,5.0,3.0,6.0,M. Tanaka,4.0,5.0,5.0,6.0,2.0,D. Hutchison,6.0,1.0,3.0,3.0,2.0,0.0,1.0,0.0
2,20150406,Tigers,Twins,4.0,0.0,3.0,0.0,10.0,5.0,D. Price,8.2,0.0,5.0,5.0,0.0,P. Hughes,6.0,4.0,8.0,6.0,1.0,1.0,0.0,0.0
3,20150406,Brewers,Rockies,0.0,10.0,0.0,10.0,8.0,16.0,K. Lohse,3.1,8.0,10.0,1.0,0.0,K. Kendrick,7.0,0.0,7.0,6.0,0.0,0.0,2.0,0.0
4,20150406,Phillies,R.Sox,0.0,8.0,0.0,4.0,3.0,9.0,C. Hamels,5.0,4.0,5.0,6.0,3.0,C. Buchholz,7.0,0.0,3.0,9.0,1.0,0.0,4.0,0.0
5,20150406,Phillies,R.Sox,0.0,8.0,0.0,4.0,3.0,9.0,C. Hamels,5.0,4.0,5.0,6.0,3.0,C. Buchholz,7.0,0.0,3.0,9.0,1.0,0.0,4.0,0.0
6,20150406,Rays,Orioles,2.0,6.0,0.0,3.0,8.0,8.0,C. Archer,5.2,4.0,6.0,5.0,1.0,C. Tillman,6.2,1.0,4.0,4.0,3.0,0.0,2.0,1.0
7,20150406,Rays,Orioles,2.0,6.0,0.0,3.0,8.0,8.0,C. Archer,5.2,4.0,6.0,5.0,1.0,C. Tillman,6.2,1.0,4.0,4.0,3.0,0.0,2.0,1.0
8,20150406,Nationals,Mets,1.0,3.0,1.0,0.0,3.0,5.0,M. Scherzer,7.2,3.0,4.0,8.0,2.0,B. Colon,6.0,1.0,3.0,8.0,1.0,0.0,0.0,0.0
9,20150406,Royals,White,10.0,1.0,4.0,0.0,13.0,5.0,Y. Ventura,6.0,1.0,4.0,2.0,1.0,J. Samardzija,6.0,5.0,6.0,1.0,3.0,1.0,0.0,5.0


# Defining data trasnformation functions (past data)

# Average of last 5 games

In [2]:
#Declaring names and variables
def past_game(nrow,df,lim,left,right,variable_to_sum,newcolumn,data_range_left,data_range_right):
    
    
        #Limiting data to  the row number
        df_filterperro= df[:lim]
        
        #Getting a dataframe with just the rows with team name  or pitcher indicated
        df_filterperro= df_filterperro.loc[(df[left].str.contains(df_set.loc[nrow,right]))]
        
        #Selecting just the last n games
        df_superdf = df_filterperro.iloc[data_range_left:data_range_right] 
        
        #Adding the last games variables
        sum1 = df_superdf[variable_to_sum].sum()
        #Getting the lenght of the filtered dataframe
        lentot = len(df_superdf) 
        
        #If there is not previous data, put a NaN, else get the average per game
        if lentot == 0:
            avg = "NaN"
        else:
            avg = sum1/ lentot
            
            
        #Save the data in the respective row and column in the new dataframe

        df_set.loc[nrow,newcolumn] = avg  
        

# Total last 5 games

In [3]:
def total_past_game(nrow,df,lim,left,right,variable_to_sum,newcolumn,data_range_left,data_range_right):
    df_filterperro= df[:lim]
        
    df_filterperro= df_filterperro.loc[(df[left].str.contains(df_set.loc[nrow,right]))]
        

        
    df_superdf = df_filterperro.iloc[data_range_left:data_range_right] 
        
    sum1 = df_superdf[variable_to_sum].sum()
    lentot = len(df_superdf) 
    if lentot == 0:
        avg = "NaN"
    else:
        #This time we are just leaving the total number without dividing it
        avg = math.ceil(sum1)

    df_set.loc[nrow,newcolumn] = avg  
    

# Bullpen innings function


In [4]:
def bullpen_total_past_game(nrow,df,lim,left,right,variable_to_sum,newcolumn,data_range_left,data_range_right):
    df_filterperro= df[:lim]
        
    df_filterperro= df_filterperro.loc[(df[left].str.contains(df_set.loc[nrow,right]))]
        

        
    df_superdf = df_filterperro.iloc[data_range_left:data_range_right] 
     
        
    
    #Getting the difference between pitcher and totalgame innings to determine bullpen innings
    sum1 =(9* len(df_superdf)) - (df_superdf[variable_to_sum].sum())
    
    lentot = len(df_superdf) 
    if lentot == 0:
        avg = "NaN"
    else:
        avg = math.ceil(sum1)

    df_set.loc[nrow,newcolumn] = avg  
    

# Creating the new df

In [5]:
import math
#Creating a copy of original df
df = df_viejo.copy()



#Creating and empty dataset to store the new features
df_set = pd.DataFrame()

#Initial point in the original df 
i= 5000
f=i

#Starting cycleto generate a new df with the calculated data from original Df:
while i < len(df) :
 #Creating the rows
    #Row number in the new df
    nrow = i-f
    #Limit on the original df
    lim = i+1
    
    #Passing the basic data ("Date, Team names, Pitcher names and the scores")
    df_set.loc[nrow,"Fecha"] = df.loc[i,"Fecha"]
    df_set.loc[nrow,"Local"] = df.loc[i,"Local"]
    df_set.loc[nrow,"Visit"] = df.loc[i,"Visit"]
    df_set.loc[nrow,"Local pitcher"] = df.loc[i,"Local pitcher"]
    df_set.loc[nrow,"Visit pitcher"] = df.loc[i,"Visit pitcher"]
    df_set.loc[nrow,"Local Score"] = df.loc[i,"Local Score"]
    df_set.loc[nrow,"Visit Score"] = df.loc[i,"Visit Score"]
    df_set.loc[nrow,"Local Win?"] = df.loc[i,"Local Win?"]
    
    #Using try to skip bugs
    try:
    
        #Local Team Last 5 games scored at home average
        past_game(nrow=nrow,df=df,lim=lim,left="Local",right="Local",variable_to_sum="Local Score",newcolumn="Local Last 5 scored at home"
                  ,data_range_left=-6,data_range_right=-1)
        
        #Local Team Last 5 games recieved at home average
        past_game(nrow=nrow,df=df,lim=lim,left="Local",right="Local",variable_to_sum="Visit Score",newcolumn="Local Last 5 received at home"
                  ,data_range_left=-6,data_range_right=-1)
        
        #Visit Team Last 5 games scored at visit average
        past_game(nrow=nrow,df=df,lim=lim,left="Visit",right="Visit",variable_to_sum="Visit Score",newcolumn="Visit Last 5 scored at visit"
                  ,data_range_left=-6,data_range_right=-1)
        
        #Visit Team Last 5 games recieved at visit average
        past_game(nrow=nrow,df=df,lim=lim,left="Visit",right="Visit",variable_to_sum="Local Score",newcolumn="Visit Last 5 recieved at visit"
                  ,data_range_left=-6,data_range_right=-1)
          
        #Local Pitcher Team Last 5 recieved at home 
        
        total_past_game(nrow=nrow,df=df,lim=lim,left="Local pitcher",right="Local pitcher",variable_to_sum="Local pitcher ERA",newcolumn="Local pitcher recieved last 5 games"
                  ,data_range_left=-6,data_range_right=-1)
        
        #Total Local Pitcher Team innings pitched last 5 games average
        total_past_game(nrow=nrow,df=df,lim=lim,left="Local pitcher",right="Local pitcher",variable_to_sum="Local pitcher in",newcolumn="Local pitcher in"
                  ,data_range_left=-6,data_range_right=-1)
        

        #Local Pitcher Team innings pitched average last 5 games
        past_game(nrow=nrow,df=df,lim=lim,left="Local pitcher",right="Local pitcher",variable_to_sum="Local pitcher ERA",newcolumn="Local pitcher in avg"
                  ,data_range_left=-6,data_range_right=-1)
         
        #Local Pitcher Team hits  
        total_past_game(nrow=nrow,df=df,lim=lim,left="Local pitcher",right="Local pitcher",variable_to_sum="Local pitcher Hit ERA",newcolumn="Local pitcher hits"
                  ,data_range_left=-6,data_range_right=-1)

        #Local pitcher K ERA 
        total_past_game(nrow=nrow,df=df,lim=lim,left="Local pitcher",right="Local pitcher",variable_to_sum="Local pitcher K ERA",newcolumn="Local pitcher k"
                  ,data_range_left=-6,data_range_right=-1)
       
        #Local pitcher bb ERA
        total_past_game(nrow=nrow,df=df,lim=lim,left="Local pitcher",right="Local pitcher",variable_to_sum="Local pitcher BB ERA",newcolumn="Local pitcher bb"
                  ,data_range_left=-6,data_range_right=-1)
          
        #Total local bullpen Team Last 5 recieved at home
        total_past_game(nrow=nrow,df=df,lim=lim,left="Local",right="Local",variable_to_sum="L bullpen recieved",newcolumn="Local Bullpen recieved last 5 games"
                  ,data_range_left=-6,data_range_right=-1)
        
        #Local Bullpen Team Last 5 in at home
        bullpen_total_past_game(nrow=nrow,df=df,lim=lim,left="Local",right="Local",variable_to_sum="Local pitcher in",newcolumn="Local Bullpen in"
                  ,data_range_left=-6,data_range_right=-1)
        #Visit Bullpen Team Last 5 in at home
        bullpen_total_past_game(nrow=nrow,df=df,lim=lim,left="Visit",right="Visit",variable_to_sum="Visit pitcher in",newcolumn="Visit Bullpen in"
                  ,data_range_left=-6,data_range_right=-1)

        #Visit Pitcher Team Last 5 recieved 
        total_past_game(nrow=nrow,df=df,lim=lim,left="Visit pitcher",right="Visit pitcher",variable_to_sum="Visit pitcher ERA",newcolumn="Visit pitcher recieved last 5 games"
                  ,data_range_left=-6,data_range_right=-1)
        
        #Total Visit Pitcher Team innings pitched last 5 games
        total_past_game(nrow=nrow,df=df,lim=lim,left="Visit pitcher",right="Visit pitcher",variable_to_sum="Visit pitcher in",newcolumn="Visit pitcher in"
                  ,data_range_left=-6,data_range_right=-1)
        
        
        #Visit Pitcher Team innings pitched average last 5 games
        past_game(nrow=nrow,df=df,lim=lim,left="Visit pitcher",right="Visit pitcher",variable_to_sum="Visit pitcher in",newcolumn="Visit pitcher in avg"
                  ,data_range_left=-6,data_range_right=-1)


        #Visit Pitcher Team hits
        total_past_game(nrow=nrow,df=df,lim=lim,left="Visit pitcher",right="Visit pitcher",variable_to_sum="Visit pitcher Hit ERA",newcolumn="Visit pitcher hits"
                  ,data_range_left=-6,data_range_right=-1)
            
        #Local pitcher K ERA
        total_past_game(nrow=nrow,df=df,lim=lim,left="Visit pitcher",right="Visit pitcher",variable_to_sum="Visit pitcher K ERA",newcolumn="Visit pitcher k"
                  ,data_range_left=-6,data_range_right=-1)
        
        #Visit pitcher bb ERA
        
        total_past_game(nrow=nrow,df=df,lim=lim,left="Visit pitcher",right="Visit pitcher",variable_to_sum="Visit pitcher BB ERA",newcolumn="Visit pitcher bb"
                  ,data_range_left=-6,data_range_right=-1)
        

        #Visit Bullpen Team Last 5 recieved at visit
        bullpen_total_past_game(nrow=nrow,df=df,lim=lim,left="Visit",right="Visit",variable_to_sum="V bullpen recieved",newcolumn="Visit Bullpen recieved last 5 games"
                  ,data_range_left=-6,data_range_right=-1)
      

       #----------------------------------------------------------
        #Last game forms
        
        #Local last 5 games form at local
        df_filterperro= df[:lim]
        df_filterperro= df_filterperro.loc[(df["Local"] == df_set.loc[nrow,"Local"])]
        df_superdf = df_filterperro.iloc[-6:-1:] 
        
        #Encoding the result (1 for a win, 0 for a loose)
        df_superdf.loc[df_superdf["Visit Score"] < df_superdf["Local Score"] ,  "Loc Win?"] = 1
        df_superdf.loc[df_superdf["Local Score"] < df_superdf["Visit Score"] ,  "Loc Win?"] = 0 
        df_superdf.reset_index(inplace=True)

        sum1 = df_superdf["Loc Win?"].sum()
        lentot = len(df_superdf)
        if lentot == 0:
            avg = "NaN"
        else:
            avg = sum1/ lentot
        df_set.loc[nrow,"Local Last 5 form at home"] = avg   
        

        #Visit last 5 form at visit
        df_filterperro= df[:lim]

        df_filterperro= df_filterperro.loc[(df["Visit"] == df_set.loc[nrow,"Visit"])]
        df_superdf1 = df_filterperro.iloc[-6:-1:] 
        #Encoding the result (1 for a win, 0 for a loose)
        df_superdf1.loc[df_superdf1["Visit Score"] > df_superdf1["Local Score"] ,  "Vis Win?"] = 1
        df_superdf1.loc[df_superdf1["Local Score"] > df_superdf1["Visit Score"] ,  "Vis Win?"] = 0 
        df_superdf1.reset_index(inplace=True)

        sum1 = df_superdf1["Vis Win?"].sum()
        lentot = len(df_superdf1)
        if lentot == 0:
            avg = "NaN"
        else:
            avg = sum1/ lentot
        df_set.loc[nrow,"Visit Last 5 form at visit"] = avg  

         #Last 5 direct results

        df_filterperro = pd.DataFrame()   
        df_superdf = pd.DataFrame()  
        try:
            df_filterperro= df[:lim]
            
            #Looking for the last head tohead matches
            df_filterperro= df_filterperro.loc[(df["Visit"] == df_set.loc[nrow,"Visit"]) & (df["Local"] == df_set.loc[nrow,"Local"])]
            df_superdf = df_filterperro.iloc[-6:-1:] 
            df_superdf.loc[df_superdf["Visit Score"] < df_superdf["Local Score"] ,  "Local Win?"] = 1
            df_superdf.loc[df_superdf["Local Score"] < df_superdf["Visit Score"] ,  "Local Win?"] = 0 
            df_superdf.reset_index(inplace=True)

            sum1 = df_superdf["Local Win?"].sum()
            lentot = len(df_superdf)
            if lentot == 0:
                avg = "NaN"
            else:
                avg = sum1/ lentot
            df_set.loc[nrow,"Direct local win?"] = avg  
        except Exception as e:
                    print("saltar dato raro",e , "Local:", df_set.loc[nrow,"Local"] ,"Visit",  df_set.loc[nrow,"Visit"]  )


        #Last 10 games local form 
        df_filterperro= df[:lim]
        
        #Looking for all general last games of local team

        df_filterperro= df_filterperro.loc[(df["Local"] == df_set.loc[nrow,"Local"])  |  (df["Visit"] == df_set.loc[nrow,"Local"])]
        df_superdf4 = df_filterperro.iloc[-11:-1:] 
        df_superdf4.reset_index(inplace=True)

        #Encoding if the local team won the last games
        for v in range (len(df_superdf4)):
            if df_superdf4.loc[v,"Visit"] == df_set.loc[nrow,"Local"]:
                if df_superdf4.loc[v,"Visit Score"]> df_superdf4.loc[v,"Local Score"]:
                    df_superdf4.loc[v,"win"]= 1 
                else: 
                    df_superdf4.loc[v,"win"]= 0
            else:
                if df_superdf4.loc[v,"Local Score"]> df_superdf4.loc[v,"Visit Score"]:
                    df_superdf4.loc[v,"win"]= 1 
                else: 
                    df_superdf4.loc[v,"win"]= 0



        sum1 = df_superdf4["win"].sum()
        lentot = len(df_superdf4)
        if lentot == 0:
            avg = "NaN"
        else:
            avg = sum1/ lentot
        df_set.loc[nrow,"Local last 10 form"] = avg  

         #Last 10 games visit form 

        df_filterperro = pd.DataFrame()
        df_filterperro= df[:lim]
        
        #Looking for all general last games of visit team

        df_filterperro= df_filterperro.loc[(df["Local"] == df_set.loc[nrow,"Visit"])  |  (df["Visit"] == df_set.loc[nrow,"Visit"])]
        df_superdf3 = df_filterperro.iloc[-11:-1:] 
        df_superdf3.reset_index(inplace=True)

        #Encoding if the visit team won the last games
        for v in range (len(df_superdf3)):
            if df_superdf3.loc[v,"Visit"] == df_set.loc[nrow,"Visit"]:
                if df_superdf3.loc[v,"Visit Score"] > df_superdf3.loc[v,"Local Score"]:
                    df_superdf3.loc[v,"winv"]= 1 
                else: 
                    df_superdf3.loc[v,"winv"]= 0
            elif df_superdf3.loc[v,"Local"] == df_set.loc[nrow,"Visit"]:
                if df_superdf3.loc[v,"Local Score"] > df_superdf3.loc[v,"Visit Score"]:
                    df_superdf3.loc[v,"winv"]= 1 
                else: 
                    df_superdf3.loc[v,"winv"]= 0



        sum1 = df_superdf3["winv"].sum()
        lentot = len(df_superdf3)
        if lentot == 0:
            avg = "NaN"
        else:
            avg = sum1/ lentot
        df_set.loc[nrow,"Visit last 10 form"] = avg  
    
    except Exception as e:
        pass



    #Printing iteration number
    i = i + 1
    print("Itr",nrow)

    
#Dropping. na and reset index   
df_set.reset_index(inplace=True)
df_set.drop("index", axis=1,inplace=True ) 
#df_set.drop(0,axis=0,inplace= True)

df_set




/Users/kike/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/kike/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Itr 0
Itr 1
Itr 2
Itr 3
Itr 4
Itr 5
Itr 6
Itr 7
Itr 8
Itr 9
Itr 10
Itr 11
Itr 12
Itr 13
Itr 14
Itr 15
Itr 16
Itr 17
Itr 18
Itr 19
Itr 20
Itr 21
Itr 22
Itr 23
Itr 24
Itr 25
Itr 26
Itr 27
Itr 28
Itr 29
Itr 30
Itr 31
Itr 32
Itr 33
Itr 34
Itr 35
Itr 36
Itr 37
Itr 38
Itr 39
Itr 40
Itr 41
Itr 42
Itr 43
Itr 44
Itr 45
Itr 46
Itr 47
Itr 48
Itr 49
Itr 50
Itr 51
Itr 52
Itr 53
Itr 54
Itr 55
Itr 56
Itr 57
Itr 58
Itr 59
Itr 60
Itr 61
Itr 62
Itr 63
Itr 64
Itr 65
Itr 66
Itr 67
Itr 68
Itr 69
Itr 70
Itr 71
Itr 72
Itr 73
Itr 74
Itr 75
Itr 76
Itr 77
Itr 78
Itr 79
Itr 80
Itr 81
Itr 82
Itr 83
Itr 84
Itr 85
Itr 86
Itr 87
Itr 88
Itr 89
Itr 90
Itr 91
Itr 92
Itr 93
Itr 94
Itr 95
Itr 96
Itr 97
Itr 98
Itr 99
Itr 100
Itr 101
Itr 102
Itr 103
Itr 104
Itr 105
Itr 106
Itr 107
Itr 108
Itr 109
Itr 110
Itr 111
Itr 112
Itr 113
Itr 114
Itr 115
Itr 116
Itr 117
Itr 118
Itr 119
Itr 120
Itr 121
Itr 122
Itr 123
Itr 124
Itr 125
Itr 126
Itr 127
Itr 128
Itr 129
Itr 130
Itr 131
Itr 132
Itr 133
Itr 134
Itr 135
Itr 136
Itr 137
Itr 13

Itr 1034
Itr 1035
Itr 1036
Itr 1037
Itr 1038
Itr 1039
Itr 1040
Itr 1041
Itr 1042
Itr 1043
Itr 1044
Itr 1045
Itr 1046
Itr 1047
Itr 1048
Itr 1049
Itr 1050
Itr 1051
Itr 1052
Itr 1053
Itr 1054
Itr 1055
Itr 1056
Itr 1057
Itr 1058
Itr 1059
Itr 1060
Itr 1061
Itr 1062
Itr 1063
Itr 1064
Itr 1065
Itr 1066
Itr 1067
Itr 1068
Itr 1069
Itr 1070
Itr 1071
Itr 1072
Itr 1073
Itr 1074
Itr 1075
Itr 1076
Itr 1077
Itr 1078
Itr 1079
Itr 1080
Itr 1081
Itr 1082
Itr 1083
Itr 1084
Itr 1085
Itr 1086
Itr 1087
Itr 1088
Itr 1089
Itr 1090
Itr 1091
Itr 1092
Itr 1093
Itr 1094
Itr 1095
Itr 1096
Itr 1097
Itr 1098
Itr 1099
Itr 1100
Itr 1101
Itr 1102
Itr 1103
Itr 1104
Itr 1105
Itr 1106
Itr 1107
Itr 1108
Itr 1109
Itr 1110
Itr 1111
Itr 1112
Itr 1113
Itr 1114
Itr 1115
Itr 1116
Itr 1117
Itr 1118
Itr 1119
Itr 1120
Itr 1121
Itr 1122
Itr 1123
Itr 1124
Itr 1125
Itr 1126
Itr 1127
Itr 1128
Itr 1129
Itr 1130
Itr 1131
Itr 1132
Itr 1133
Itr 1134
Itr 1135
Itr 1136
Itr 1137
Itr 1138
Itr 1139
Itr 1140
Itr 1141
Itr 1142
Itr 1143
Itr 1144
I

Itr 1945
Itr 1946
Itr 1947
Itr 1948
Itr 1949
Itr 1950
Itr 1951
Itr 1952
Itr 1953
Itr 1954
Itr 1955
Itr 1956
Itr 1957
Itr 1958
Itr 1959
Itr 1960
Itr 1961
Itr 1962
Itr 1963
Itr 1964
Itr 1965
Itr 1966
Itr 1967
Itr 1968
Itr 1969
Itr 1970
Itr 1971
Itr 1972
Itr 1973
Itr 1974
Itr 1975
Itr 1976
Itr 1977
Itr 1978
Itr 1979
Itr 1980
Itr 1981
Itr 1982
Itr 1983
Itr 1984
Itr 1985
Itr 1986
Itr 1987
Itr 1988
Itr 1989
Itr 1990
Itr 1991
Itr 1992
Itr 1993
Itr 1994
Itr 1995
Itr 1996
Itr 1997
Itr 1998
Itr 1999
Itr 2000
Itr 2001
Itr 2002
Itr 2003
Itr 2004
Itr 2005
Itr 2006
Itr 2007
Itr 2008
Itr 2009
Itr 2010
Itr 2011
Itr 2012
Itr 2013
Itr 2014
Itr 2015
Itr 2016
Itr 2017
Itr 2018
Itr 2019
Itr 2020
Itr 2021
Itr 2022
Itr 2023
Itr 2024
Itr 2025
Itr 2026
Itr 2027
Itr 2028
Itr 2029
Itr 2030
Itr 2031
Itr 2032
Itr 2033
Itr 2034
Itr 2035
Itr 2036
Itr 2037
Itr 2038
Itr 2039
Itr 2040
Itr 2041
Itr 2042
Itr 2043
Itr 2044
Itr 2045
Itr 2046
Itr 2047
Itr 2048
Itr 2049
Itr 2050
Itr 2051
Itr 2052
Itr 2053
Itr 2054
Itr 2055
I

Itr 2856
Itr 2857
Itr 2858
Itr 2859
Itr 2860
Itr 2861
Itr 2862
Itr 2863
Itr 2864
Itr 2865
Itr 2866
Itr 2867
Itr 2868
Itr 2869
Itr 2870
Itr 2871
Itr 2872
Itr 2873
Itr 2874
Itr 2875
Itr 2876
Itr 2877
Itr 2878
Itr 2879
Itr 2880
Itr 2881
Itr 2882
Itr 2883
Itr 2884
Itr 2885
Itr 2886
Itr 2887
Itr 2888
Itr 2889
Itr 2890
Itr 2891
Itr 2892
Itr 2893
Itr 2894
Itr 2895
Itr 2896
Itr 2897
Itr 2898
Itr 2899
Itr 2900
Itr 2901
Itr 2902
Itr 2903
Itr 2904
Itr 2905
Itr 2906
Itr 2907
Itr 2908
Itr 2909
Itr 2910
Itr 2911
Itr 2912
Itr 2913
Itr 2914
Itr 2915
Itr 2916
Itr 2917
Itr 2918
Itr 2919
Itr 2920
Itr 2921
Itr 2922
Itr 2923
Itr 2924
Itr 2925
Itr 2926
Itr 2927
Itr 2928
Itr 2929
Itr 2930
Itr 2931
Itr 2932
Itr 2933
Itr 2934
Itr 2935
Itr 2936
Itr 2937
Itr 2938
Itr 2939
Itr 2940
Itr 2941
Itr 2942
Itr 2943
Itr 2944
Itr 2945
Itr 2946
Itr 2947
Itr 2948
Itr 2949
Itr 2950
Itr 2951
Itr 2952
Itr 2953
Itr 2954
Itr 2955
Itr 2956
Itr 2957
Itr 2958
Itr 2959
Itr 2960
Itr 2961
Itr 2962
Itr 2963
Itr 2964
Itr 2965
Itr 2966
I

...

Itr 9233
Itr 9234
Itr 9235
Itr 9236
Itr 9237
Itr 9238
Itr 9239
Itr 9240
Itr 9241
Itr 9242
Itr 9243
Itr 9244
Itr 9245
Itr 9246
Itr 9247
Itr 9248
Itr 9249
Itr 9250
Itr 9251
Itr 9252
Itr 9253
Itr 9254
Itr 9255
Itr 9256
Itr 9257
Itr 9258
Itr 9259
Itr 9260
Itr 9261
Itr 9262
Itr 9263
Itr 9264
Itr 9265
Itr 9266
Itr 9267
Itr 9268
Itr 9269
Itr 9270
Itr 9271
Itr 9272
Itr 9273
Itr 9274
Itr 9275
Itr 9276
Itr 9277
Itr 9278
Itr 9279
Itr 9280
Itr 9281
Itr 9282
Itr 9283
Itr 9284
Itr 9285
Itr 9286
Itr 9287
Itr 9288
Itr 9289
Itr 9290
Itr 9291
Itr 9292
Itr 9293
Itr 9294
Itr 9295
Itr 9296
Itr 9297
Itr 9298
Itr 9299
Itr 9300
Itr 9301
Itr 9302
Itr 9303
Itr 9304
Itr 9305
Itr 9306
Itr 9307
Itr 9308
Itr 9309
Itr 9310
Itr 9311
Itr 9312
Itr 9313
Itr 9314
Itr 9315
Itr 9316
Itr 9317
Itr 9318
Itr 9319
Itr 9320
Itr 9321
Itr 9322
Itr 9323
Itr 9324
Itr 9325
Itr 9326
Itr 9327
Itr 9328
Itr 9329
Itr 9330
Itr 9331
Itr 9332
Itr 9333
Itr 9334
Itr 9335
Itr 9336
Itr 9337
Itr 9338
Itr 9339
Itr 9340
Itr 9341
Itr 9342
Itr 9343
I

Itr 10129
Itr 10130
Itr 10131
Itr 10132
Itr 10133
Itr 10134
Itr 10135
Itr 10136
Itr 10137
Itr 10138
Itr 10139
Itr 10140
Itr 10141
Itr 10142
Itr 10143
Itr 10144
Itr 10145
Itr 10146
Itr 10147
Itr 10148
Itr 10149
Itr 10150
Itr 10151
Itr 10152
Itr 10153
Itr 10154
Itr 10155
Itr 10156
Itr 10157
Itr 10158
Itr 10159
Itr 10160
Itr 10161
Itr 10162
Itr 10163
Itr 10164
Itr 10165
Itr 10166
Itr 10167
Itr 10168
Itr 10169
Itr 10170
Itr 10171
Itr 10172
Itr 10173
Itr 10174
Itr 10175
Itr 10176
Itr 10177
Itr 10178
Itr 10179
Itr 10180
Itr 10181
Itr 10182
Itr 10183
Itr 10184
Itr 10185
Itr 10186
Itr 10187
Itr 10188
Itr 10189
Itr 10190
Itr 10191
Itr 10192
Itr 10193
Itr 10194
Itr 10195
Itr 10196
Itr 10197
Itr 10198
Itr 10199
Itr 10200
Itr 10201
Itr 10202
Itr 10203
Itr 10204
Itr 10205
Itr 10206
Itr 10207
Itr 10208
Itr 10209
Itr 10210
Itr 10211
Itr 10212
Itr 10213
Itr 10214
Itr 10215
Itr 10216
Itr 10217
Itr 10218
Itr 10219
Itr 10220
Itr 10221
Itr 10222
Itr 10223
Itr 10224
Itr 10225
Itr 10226
Itr 10227
Itr 10228


Itr 10949
Itr 10950
Itr 10951
Itr 10952
Itr 10953
Itr 10954
Itr 10955
Itr 10956
Itr 10957
Itr 10958
Itr 10959
Itr 10960
Itr 10961
Itr 10962
Itr 10963
Itr 10964
Itr 10965
Itr 10966
Itr 10967
Itr 10968
Itr 10969
Itr 10970
Itr 10971
Itr 10972
Itr 10973
Itr 10974
Itr 10975
Itr 10976
Itr 10977
Itr 10978
Itr 10979
Itr 10980
Itr 10981
Itr 10982
Itr 10983
Itr 10984
Itr 10985
Itr 10986
Itr 10987
Itr 10988
Itr 10989
Itr 10990
Itr 10991
Itr 10992
Itr 10993
Itr 10994
Itr 10995
Itr 10996
Itr 10997
Itr 10998
Itr 10999
Itr 11000
Itr 11001
Itr 11002
Itr 11003
Itr 11004
Itr 11005
Itr 11006
Itr 11007
Itr 11008
Itr 11009
Itr 11010
Itr 11011
Itr 11012
Itr 11013
Itr 11014
Itr 11015
Itr 11016
Itr 11017
Itr 11018
Itr 11019
Itr 11020
Itr 11021
Itr 11022
Itr 11023
Itr 11024
Itr 11025
Itr 11026
Itr 11027
Itr 11028
Itr 11029
Itr 11030
Itr 11031
Itr 11032
Itr 11033
Itr 11034
Itr 11035
Itr 11036
Itr 11037
Itr 11038
Itr 11039
Itr 11040
Itr 11041
Itr 11042
Itr 11043
Itr 11044
Itr 11045
Itr 11046
Itr 11047
Itr 11048


Itr 11769
Itr 11770
Itr 11771
Itr 11772
Itr 11773
Itr 11774
Itr 11775
Itr 11776
Itr 11777
Itr 11778
Itr 11779
Itr 11780
Itr 11781
Itr 11782
Itr 11783
Itr 11784
Itr 11785
Itr 11786
Itr 11787
Itr 11788
Itr 11789
Itr 11790
Itr 11791
Itr 11792
Itr 11793
Itr 11794
Itr 11795
Itr 11796
Itr 11797
Itr 11798
Itr 11799
Itr 11800
Itr 11801
Itr 11802
Itr 11803
Itr 11804
Itr 11805
Itr 11806
Itr 11807
Itr 11808
Itr 11809
Itr 11810
Itr 11811
Itr 11812
Itr 11813
Itr 11814
Itr 11815
Itr 11816
Itr 11817
Itr 11818
Itr 11819
Itr 11820
Itr 11821
Itr 11822
Itr 11823
Itr 11824
Itr 11825
Itr 11826
Itr 11827
Itr 11828
Itr 11829
Itr 11830
Itr 11831
Itr 11832
Itr 11833
Itr 11834
Itr 11835
Itr 11836
Itr 11837
Itr 11838
Itr 11839
Itr 11840
Itr 11841
Itr 11842
Itr 11843
Itr 11844
Itr 11845
Itr 11846
Itr 11847
Itr 11848
Itr 11849
Itr 11850
Itr 11851
Itr 11852
Itr 11853
Itr 11854
Itr 11855
Itr 11856
Itr 11857
Itr 11858
Itr 11859
Itr 11860
Itr 11861
Itr 11862
Itr 11863
Itr 11864
Itr 11865
Itr 11866
Itr 11867
Itr 11868


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kike/Python/PROYECTOS/Github_LocalWinBaseball/sets/mlb_dataset_treated.csv'

In [8]:
#Saving resulting df to a csv
path = pathlib.Path.cwd() / 'sets' / 'mlb_dataset_treated.csv'
df_set.to_csv(path, index= False)

df_set


,Fecha,Local,Visit,Local pitcher,Visit pitcher,Local Score,Visit Score,Local Win?,Local Last 5 scored at home,Local Last 5 received at home,Visit Last 5 scored at visit,Visit Last 5 recieved at visit,Local pitcher recieved last 5 games,Local pitcher in,Local pitcher in avg,Local pitcher hits,Local pitcher k,Local pitcher bb,Local Bullpen recieved last 5 games,Local Bullpen in,Visit Bullpen in,Visit pitcher recieved last 5 games,Visit pitcher in,Visit pitcher in avg,Visit pitcher hits,Visit pitcher k,Visit pitcher bb,Visit Bullpen recieved last 5 games,Local Last 5 form at home,Visit Last 5 form at visit,Direct local win?,Local last 10 form,Visit last 10 form
0,20170419.0,Mariners,Marlins,F. Hernandez,E. Volquez,10.0,5.0,1.0,4.2,2.8,4.6,2.8,18.0,28.0,3.6,32.0,20.0,8.0,3.0,14.0,17.0,25.0,28.0,5.6,35.0,20.0,8.0,43.0,0.8,0.6,0.5,0.5,0.6
1,20170419.0,Yankees,White,M. Tanaka,D. Covey,9.0,1.0,1.0,4.8,3.2,2.6,3.2,11.0,33.0,2.2,28.0,35.0,5.0,3.0,11.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,44.0,0.8,0.6,0.6,0.8,0.6
2,20170419.0,Blue,R.Sox,F. Liriano,R. Porcello,3.0,0.0,1.0,3.6,5.6,4.4,4.8,18.0,26.0,3.6,26.0,30.0,14.0,14.0,19.0,16.0,11,37,7.22,26,33,4,39.0,0.2,0.4,0.2,0.1,0.7
3,20170419.0,Reds,Orioles,A. Garrett,U. Jimenez,0.0,2.0,0.0,4.6,5.4,4.6,4.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,25.0,19.0,12,33,6.46,18,22,10,38.0,0.4,0.6,1.0,0.6,0.6
4,20170419.0,Mets,Phillies,R. Gsellman,V. Velasquez,5.0,4.0,1.0,3.2,5.0,4.0,4.0,10,30,2.0,27,30,10,12.0,18.0,17.0,18,28,5.44,38,29,11,37.0,0.4,0.4,0.6,0.5,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11942,20220717.0,Cubs,Mets,A. Sampson,D. Peterson,3.0,2.0,1.0,1.4,5.0,4.4,2.2,16,26,3.2,30,15,8,10.0,22.0,16.0,13,25,4.9,15,36,13,40.0,0.0,0.8,0.4,0.1,0.7
11943,20220717.0,Rangers,Mariners,G. Otto,C. Flexen,2.0,6.0,0.0,4.4,6.6,5.0,3.0,25,21,5.0,27,16,19,26.0,23.0,20.0,9,29,5.68,26,17,11,40.0,0.2,1.0,0.2,0.4,1.0
11944,20220717.0,Rockies,Pirates,A. Gomber,B. Wilson,3.0,8.0,0.0,7.6,3.2,2.2,5.0,21,28,4.2,38,16,4,5.0,19.0,19.0,19,24,4.64,29,14,6,29.0,1.0,0.2,1.0,0.8,0.4
11945,20220717.0,Giants,Brewers,L. Webb,A. Ashby,9.0,5.0,1.0,5.8,2.4,3.2,3.8,9,31,1.8,25,31,9,7.0,22.0,18.0,19,23,4.48,33,27,9,34.0,0.8,0.4,0.6,0.6,0.3
